In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
DATA='EURUSD'

# extended_columns = [str(num)+'h' for num in np.arange(1,25)]
# extended_columns
# names=['time','currency','description','impact','actual','deviation','consensus','previous','result'].extend()

['1h',
 '2h',
 '3h',
 '4h',
 '5h',
 '6h',
 '7h',
 '8h',
 '9h',
 '10h',
 '11h',
 '12h',
 '13h',
 '14h',
 '15h',
 '16h',
 '17h',
 '18h',
 '19h',
 '20h',
 '21h',
 '22h',
 '23h',
 '24h']

In [15]:
frame = pd.read_csv('./data/economic_events_'+DATA+'.csv',header=0,parse_dates=['time'],index_col='time',date_format='%y-%m-%d %H:%M:%S%Z')

In [16]:
frame.head()

,currency,description,impact,actual,deviation,consensus,previous,hex_color_actual,1h,2h,...,15h,16h,17h,18h,19h,20h,21h,22h,23h,24h
time,,,,,,,,,,,,,,,,,,,,,
2013-01-02 09:13:00+00:00,EUR,HCOB Manufacturing PMI(Dec),low,44.6,-0.48,45.1,45.3,red,1.32448,1.32479,...,1.31821,1.31703,1.31267,1.31387,1.31371,1.31382,1.31452,1.31339,1.31636,1.3147
2013-01-02 09:43:00+00:00,EUR,HCOB Manufacturing PMI(Dec),low,46.7,1.20,45.3,45.1,green,1.32448,1.32479,...,1.31821,1.31703,1.31267,1.31387,1.31371,1.31382,1.31452,1.31339,1.31636,1.3147
2013-01-02 09:48:00+00:00,EUR,HCOB Manufacturing PMI(Dec),low,44.6,0.00,44.6,44.6,black,1.32448,1.32479,...,1.31821,1.31703,1.31267,1.31387,1.31371,1.31382,1.31452,1.31339,1.31636,1.3147
2013-01-02 09:53:00+00:00,EUR,HCOB Manufacturing PMI(Dec),medium,46,-0.28,46.3,46.3,red,1.32448,1.32479,...,1.31821,1.31703,1.31267,1.31387,1.31371,1.31382,1.31452,1.31339,1.31636,1.3147
2013-01-02 09:58:00+00:00,EUR,HCOB Manufacturing PMI(Dec),medium,46.1,-0.32,46.3,46.3,red,1.32448,1.32479,...,1.31821,1.31703,1.31267,1.31387,1.31371,1.31382,1.31452,1.31339,1.31636,1.3147


In [94]:
frame.shape

(372768, 8)

In [95]:
frame.describe()

,currency,description,impact,actual,deviation,consensus,previous,result
count,372768,372768,372768,372768,372768,372768,372768,372768
unique,34,7059,4,11095,1815,8980,11080,3
top,EUR,Consumer Price Index (YoY)(Jan),low,0.2%,0.00,0.2%,0.2%,green
freq,102150,1032,209052,9102,64566,12978,9102,157974


In [96]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 372768 entries, 2013-01-01 02:00:00 to 2024-05-24 15:00:00
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   currency     372768 non-null  object
 1   description  372768 non-null  object
 2   impact       372768 non-null  object
 3   actual       372768 non-null  object
 4   deviation    372768 non-null  object
 5   consensus    372768 non-null  object
 6   previous     372768 non-null  object
 7   result       372768 non-null  object
dtypes: object(8)
memory usage: 25.6+ MB


In [97]:
frame.deviation.value_counts()

deviation
0.00     64566
-         5406
-2.04     1926
-2.50     1848
-1.34     1848
         ...  
4.92         6
6.76         6
14.25        6
10.87        6
7.06         6
Name: count, Length: 1815, dtype: int64

In [98]:
frame['deviation'] = frame.deviation.replace('-',None).astype(np.float64)

In [99]:
frame.deviation.value_counts().tail()

deviation
 16.64    6
-23.19    6
 8.20     6
 92.42    6
 7.06     6
Name: count, dtype: int64

In [103]:
frame.result.value_counts()

result
green    157974
red      152778
black     62016
Name: count, dtype: int64